In [262]:
import torch
from torchtext.vocab import vocab
from torch import nn
import os 
import sys
from box_embeddings.modules.volume.volume import Volume
from box_embeddings.modules.intersection import Intersection
from utils.model import BoxModel
sys.path.append("../")
model = torch.load("/home/fmollica/Desktop/word2box_pytorch/weights/skipgram_WikiText2/epochs_2_min_count_50_batch_size_512_embed_dim_3_lr_0.001_window_10_neg_count_10/model_final.pt")


In [263]:
model

BoxModel(
  (embeddings_word): BoxEmbedding(3520, 6)
  (embeddings_context): BoxEmbedding(3520, 6)
  (box_vol): Volume()
  (box_int): Intersection()
  (vocab): Vocab()
)

In [264]:
def most_similar_2(word, N=None):

        if N is None:
            N = len(model.vocab)

        embedding_all_target = model.embeddings_word.all_boxes

        
        index_word = (model.vocab[word])
        print(index_word)
        embedding_word = embedding_all_target[index_word]
                
        # sim3 = torch.exp(self.box_vol(self.box_int(embedding_all_target, embedding_word))-torch.minimum(self.box_vol(embedding_all_target), self.box_vol(embedding_word)))
            
        # idx = (-sim3).argsort()
        

        sim3 = torch.exp(model.box_vol(model.box_int(embedding_all_target, embedding_word))- torch.maximum(model.box_vol(embedding_word),model.box_vol(embedding_all_target)))

        idx = (-sim3).argsort()

        print(idx[0:50])

        for i, index in enumerate(idx[0:50]):
            print("Similar to : ", str(index.item()),   model.vocab.lookup_token(index))

In [265]:
uno = model.embeddings_word.all_boxes[143]
due = model.embeddings_word.all_boxes[143]
vol = Volume(volume_temperature=0.1, intersection_temperature=0.0001)
inte = Intersection(intersection_temperature=0)

print(torch.exp(vol(inte(uno, due))).item())
print("Uno " , torch.exp(vol(uno)).item(), " Due " , torch.exp(vol(due)).item())

print(torch.exp(vol(inte(uno, due)) -  torch.maximum(vol(uno), vol(due))).item())

uno = model.embeddings_word.all_boxes[143]
due = model.embeddings_word.all_boxes[126]
print(torch.exp(vol(inte(uno, due))).item())
print("Uno " , torch.exp(vol(uno)), " Due " , torch.exp(vol(due)))
print(torch.exp(vol(inte(uno, due)) -  torch.maximum(vol(uno), vol(due))).item())



0.0007327903877012432
Uno  0.0007327903877012432  Due  0.0007327903877012432
1.0
0.0006932641845196486
Uno  tensor(0.0007, device='cuda:0', grad_fn=<ExpBackward0>)  Due  tensor(0.0078, device='cuda:0', grad_fn=<ExpBackward0>)
0.08904954791069031


In [268]:
most_similar_2("war")

32
tensor([  32, 2139, 1316,  928, 1637,  387, 1001, 2959,  353,   97, 1421,  852,
         475, 1549, 1081, 3010,  529, 2386,  311,  252, 1289,  447, 1260,  669,
         418,  245, 2290, 3469,  191, 1628,  996,  456,  596,  312, 3272, 2825,
         831,   67, 1879,  775,  249,  439,  496, 1539, 3192,  863,  877,  433,
         329, 2442], device='cuda:0')
Similar to :  32 war
Similar to :  2139 example
Similar to :  1316 prominent
Similar to :  928 including
Similar to :  1637 album
Similar to :  387 could
Similar to :  1001 market
Similar to :  2959 slam
Similar to :  353 stated
Similar to :  97 part
Similar to :  1421 minute
Similar to :  852 paper
Similar to :  475 sold
Similar to :  1549 ninth
Similar to :  1081 described
Similar to :  3010 soviet
Similar to :  529 include
Similar to :  2386 isle
Similar to :  311 however
Similar to :  252 woman
Similar to :  1289 male
Similar to :  447 duty
Similar to :  1260 dark
Similar to :  669 artillery
Similar to :  418 seven
Similar to :